In [57]:
from collections import namedtuple

ALIVE = '*'
EMPTY = '-'
TICK = object()

Query = namedtuple('Query', 'y x')

Transition = namedtuple('Transition', 'y x state')


def count_neighbors(y, x):
    n_ = yield Query(y + 1, x + 0)  # North
    ne = yield Query(y + 1, x + 1)  # Northeast
    e_ = yield Query(y + 0, x + 1)  # East
    se = yield Query(y - 1, x + 1)  # Southeast
    s_ = yield Query(y - 1, x + 0)  # South
    sw = yield Query(y - 1, x - 1)  # Southwest
    w_ = yield Query(y + 0, x - 1)  # West
    nw = yield Query(y + 1, x - 1)  # Northwest
    neighbor_states = [n_, ne, e_, se, s_, sw, w_, nw]
    count = 0
    for state in neighbor_states:
        if state == ALIVE:
            count += 1
    return count


def game_logic(state, neighbors):
    if state == ALIVE:
        if neighbors < 2:
            return EMPTY     # Die: Too few
        elif neighbors > 3:
            return EMPTY     # Die: Too many
    else:
        if neighbors == 3:
            return ALIVE     # Regenerate
    return state


def step_cell(y, x):
    state = yield Query(y, x)
    neighbors = yield from count_neighbors(y, x)
    next_state = game_logic(state, neighbors)
    yield Transition(y, x, next_state)


def simulate(height, width):
    while True:
        for y in range(height):
            for x in range(width):
                yield from step_cell(y, x)
        yield TICK


class Grid(object):
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = []
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)

    def __str__(self):
        output = ''
        for row in self.rows:
            for cell in row:
                output += cell
            output += '\n'
        return output

    def __getitem__(self, position):
        y, x = position
        return self.rows[y % self.height][x % self.width]

    def __setitem__(self, position, state):
        y, x = position
        self.rows[y % self.height][x % self.width] = state


def live_a_generation(grid, sim):
    progeny = Grid(grid.height, grid.width)
    item = next(sim)
    while item is not TICK:
        if isinstance(item, Query):
            state = grid[item.y, item.x]
            item = sim.send(state)
        else:  # Must be a Transition
            progeny[item.y, item.x] = item.state
            item = next(sim)
    return progeny


class ColumnPrinter(object):
    def __init__(self):
        self.columns = []

    def append(self, data):
        self.columns.append(data)

    def __str__(self):
        row_count = 1
        for data in self.columns:
            row_count = max(row_count, len(data.splitlines()) + 1)
        rows = [''] * row_count
        for j in range(row_count):
            for i, data in enumerate(self.columns):
                line = data.splitlines()[max(0, j - 1)]
                if j == 0:
                    rows[j] += str(i).center(len(line))
                else:
                    rows[j] += line
                if (i + 1) < len(self.columns):
                    rows[j] += ' | '
        return '\n'.join(rows)

In [58]:
it = count_neighbors(10, 5)
next(it)#Get the first query, for q1

Query(y=11, x=5)

In [59]:
it.send(ALIVE)# Send q1 state, get q2

Query(y=11, x=6)

In [60]:
it.send(ALIVE)# Send q2 state, get q3

Query(y=10, x=6)

In [61]:
[it.send(state) for state in (EMPTY)*5]

[Query(y=9, x=6),
 Query(y=9, x=5),
 Query(y=9, x=4),
 Query(y=10, x=4),
 Query(y=11, x=4)]

In [62]:
try:
    it.send(EMPTY)
except StopIteration as e:
    count = e.value

count

2

In [63]:
it2 = step_cell(100, 50)

In [64]:
next(it2)

Query(y=100, x=50)

In [65]:
[it2.send(st) for st in (ALIVE)*5 + (EMPTY)*3]

[Query(y=101, x=50),
 Query(y=101, x=51),
 Query(y=100, x=51),
 Query(y=99, x=51),
 Query(y=99, x=50),
 Query(y=99, x=49),
 Query(y=100, x=49),
 Query(y=101, x=49)]

In [66]:
it2.send(EMPTY)

Transition(y=100, x=50, state='-')